In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# Đọc dữ liệu từ tệp CSV
try:
    data = pd.read_csv('../../../data/BCP.csv')
    output_file = '../../../data/BCP_predictions.csv'
    # Xem trước dữ liệu
    print("Dữ liệu gốc:")
    print(data.head())
    print(data.columns)

    # Loại bỏ cột 'Time' vì không sử dụng trong tính toán
    data = data.drop(columns=['Time'])

    # Kiểm tra giá trị NaN
    print("\nKiểm tra giá trị NaN:")
    print(data.isna().sum())

    # Xóa các hàng chứa giá trị NaN
    data.dropna(inplace=True)

    # Giả sử chúng ta sẽ chọn cột '440FI539' làm mục tiêu và các cột còn lại là đặc trưng
    target_column = '440FI539'  # Bạn có thể chọn cột khác nếu cần
    features = [col for col in data.columns if col != target_column]

    # Chuyển đổi cột mục tiêu thành nhị phân (0 hoặc 1) dựa trên giá trị trung bình
    data[f'{target_column}_binary'] = (data[target_column] > data[target_column].mean()).astype(int)

    X = data[features]
    y = data[f'{target_column}_binary']

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Huấn luyện mô hình hồi quy Logistic
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)

    # Dự đoán với mô hình hồi quy Logistic
    y_pred_logistic = logistic_model.predict(X_test)

    # Huấn luyện mô hình MLP
    mlp_model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
    mlp_model.fit(X_train, y_train)

    # Dự đoán với mô hình MLP
    y_pred_mlp = mlp_model.predict(X_test)

    # Kết hợp kết quả dự đoán từ cả hai mô hình
    y_pred_combined = (y_pred_logistic + y_pred_mlp) / 2
    y_pred_combined = np.round(y_pred_combined).astype(int)

    # Đánh giá độ chính xác và F1 score
    accuracy = accuracy_score(y_test, y_pred_combined)
    f1 = f1_score(y_test, y_pred_combined)

    # Thêm cột dự đoán vào dữ liệu ban đầu với tên cột mới
    X_test_df = pd.DataFrame(X_test, columns=features)
    data.loc[X_test_df.index, f'{target_column}_prediction'] = y_pred_combined

    data.to_csv(output_file, index=False)

    # Hiển thị kết quả
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Data Result with Predictions:\n{data.head()}")

except Exception as e:
    print(f"Error processing the data: {e}")


Dữ liệu gốc:
               Time  440FI539  440FI540  440PDI541  440PDI564  440PDIC565  \
0  28/04/2023 03:47     3.823     3.316       0.01      0.375       4.649   
1  28/04/2023 03:52     3.798     3.304       0.01      0.375       4.681   
2  28/04/2023 03:57     3.827     3.304       0.01      0.375       4.658   
3  28/04/2023 04:02     3.821     3.280       0.01      0.375       4.654   
4  28/04/2023 04:07     3.830     3.278       0.01      0.376       4.683   

   440PI546  440PI551  440PI553  440PI558A  440PI558B  440PI558C  440PI561A  
0     3.635     0.463      0.47      1.721      1.692      1.541      1.786  
1     3.635     0.464      0.47      1.719      1.690      1.539      1.781  
2     3.637     0.464      0.47      1.721      1.692      1.540      1.783  
3     3.636     0.465      0.47      1.722      1.692      1.541      1.783  
4     3.636     0.464      0.47      1.721      1.694      1.541      1.786  
Index(['Time', '440FI539', '440FI540', '440PDI541', '440